In [ ]:
"""
Working with the eBird API.

This notebook show you how to create a database and load observations
from the eBird API v2.0.

You will need to sign up for a key to access the eBird API.
https://ebird.org/api/keygen

You will also need to copy the example file .env.example to .env,
and add your API key.
"""
import datetime as dt

from sqlalchemy import create_engine

from main import settings
from main.db import models, loaders

db_dir = settings.DATABASE_DIR
db_name = "api_checklists"
db_url = f"sqlite+pysqlite:///{db_dir}/{db_name}.sqlite3"

# The key needed to access the API.
api_key = settings.API_KEY
# The regions for which to load observations for.
regions = settings.API_REGIONS
# The number of days to fetch checklists for.
back = settings.API_PAST_DAYS

In [ ]:
# Create and initialize the database.
engine = create_engine(db_url)
models.Base.metadata.create_all(engine)

In [ ]:
# The eBird API returns only the species code, e.g. "horlar1", so
# we need to populate the species table with the complete taxonomy.
loader = loaders.SpeciesLoader(api_key, db_url)
loader.load()

In [ ]:
# Load the observations from the eBird API.
loader = loaders.APILoader(api_key, db_url)

today = dt.date.today()
areas = [region.strip() for region in regions.split(",")]
dates = [today - dt.timedelta(days=offset) for offset in range(back)]

for area in areas:
    for date in dates:
        loader.load(area, date)

In [ ]:
# If you are running Linux or MacOS you can schedule the loading of
# checklists from the eBird API using cron. Here we get cron to run
# the load script at 9am, every day. Note: your crontab will be updated
# every time you execute this cell.

from crontab import CronTab
from pathlib import Path

python = Path(settings.ROOT_DIR, ".venv", "bin", "python")
script = Path(settings.ROOT_DIR, "scripts", "load.py")

with CronTab(user=True) as cron:
    job = cron.new(command=f"%s %s" % (python, script))
    job.hour.on(9)